In [ ]:
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

from regressio.models import *
from regressio.datagen import *

In [ ]:
x,y = generate_random_walk(100)

model = cubic_spline(knots=3)
model.fit(x, y, plot=True)

In [ ]:
from scipy.interpolate import CubicSpline

np.random.seed(10)

x,y = generate_random_walk(100)

x = np.arange(3)
y = np.sin(x)
cs = CubicSpline(x, y)
xs = np.arange(-0.5, 9.6, 0.1)
fig, ax = plt.subplots(figsize=(6.5, 4))
ax.plot(x, y, 'o', label='data')
ax.plot(xs, np.sin(xs), label='true')
ax.plot(xs, cs(xs), label="S")
ax.plot(xs, cs(xs, 1), label="S'")
ax.plot(xs, cs(xs, 2), label="S''")
ax.plot(xs, cs(xs, 3), label="S'''")
ax.set_xlim(-0.5, 9.5)
ax.legend(loc='lower left', ncol=2)
plt.show()

Try with 3 points.

In [ ]:
raw_x, raw_y = np.array([1,2,3]),  np.array([1,5,4])

b = np.array([1, 5, 5, 4, 0, 0, 0, 0])
A = np.array([[1,1,1,1,0,0,0,0],
              [8,4,2,1,0,0,0,0], 
              [0,0,0,0,8,4,2,1],
              [0,0,0,0,27,9,3,1],
              [12,4,1,0,-12,-4,-1,0],
              [12,2,0,0,-12,-2,0,0],
              [6,2,0,0,0,0,0,0],
              [0,0,0,0,18,2,0,0]])

lines = np.dot(np.linalg.inv(A), b)
print(lines)
lines = lines.reshape(-1, 4)
print(lines)

def plot(values, coeffs):
    expanded = np.hstack([coeffs[i] * (values ** i) for i in range(0, len(coeffs))])
    return np.sum(expanded, axis=1)

xs = np.linspace(1, 3, 100)

# Need to make this nicer, but for now this is fine
y1s = plot(xs[xs<=2].reshape(-1,1), lines[0][::-1])
y2s = plot(xs[xs>2].reshape(-1,1), lines[1][::-1])
y = np.concatenate([y1s, y2s])

plt.scatter(xs, y)
plt.scatter(raw_x, raw_y, c='tab:red')
plt.show()

Now with 4 points.

In [ ]:
raw_x, raw_y = np.array([1,2,3,4]),  np.array([1,5,4,5])

b = np.array([1, 5, 5, 4, 4, 5, 0, 0, 0, 0, 0, 0], dtype=np.float64)
A = np.array([[1,1,1,1,0,0,0,0,0,0,0,0],
              [8,4,2,1,0,0,0,0,0,0,0,0], 
              [0,0,0,0,8,4,2,1,0,0,0,0],
              [0,0,0,0,27,9,3,1,0,0,0,0],
              [0,0,0,0,0,0,0,0,27,9,3,1],
              [0,0,0,0,0,0,0,0,64,16,4,1], # functions to here
              [12,4,1,0,-12,-4,-1,0,0,0,0,0],
              [12,2,0,0,-12,-2,0,0,0,0,0,0],
              [0,0,0,0,27,6,1,0,-27,-6,-1,0],
              [0,0,0,0,18,2,0,0,-18,-2,0,0], # derivatives to here
              [6,2,0,0,0,0,0,0,0,0,0,0],
              [0,0,0,0,0,0,0,0,24,2,0,0], # endpoints to here
              ], dtype=np.float64)

lines = np.dot(np.linalg.inv(A), b)
print(lines)
lines = lines.reshape(-1, 4)

def plot(values, coeffs):
    expanded = np.hstack([coeffs[i] * (values ** i) for i in range(0, len(coeffs))])
    return np.sum(expanded, axis=1)

xs = np.linspace(1, 4, 100)

# Need to make this nicer, but for now this is fine
y1s = plot(xs[xs<=2].reshape(-1,1), lines[0][::-1])
y2s = plot(xs[np.logical_and(xs>2, xs<=3)].reshape(-1,1), lines[1][::-1])
y3s = plot(xs[xs>3].reshape(-1,1), lines[2][::-1])

ys = np.concatenate([y1s, y2s, y3s])

plt.scatter(xs, ys)
plt.scatter(raw_x, raw_y, c='tab:red')
plt.show()

Now for N points.

In [ ]:
raw_x, raw_y = np.array([1,2,3]),  np.array([1,5,4])

b = np.array([1, 5, 5, 4, 0, 0, 0, 0])
A = np.array([[1,1,1,1,0,0,0,0],
              [8,4,2,1,0,0,0,0], 
              [0,0,0,0,8,4,2,1],
              [0,0,0,0,27,9,3,1],
              [12,4,1,0,-12,-4,-1,0],
              [12,2,0,0,-12,-2,0,0],
              [6,2,0,0,0,0,0,0],
              [0,0,0,0,18,2,0,0]])

lines = np.dot(np.linalg.inv(A), b)
print(lines)
lines = lines.reshape(-1, 4)
print(lines)

def plot(values, coeffs):
    expanded = np.hstack([coeffs[i] * (values ** i) for i in range(0, len(coeffs))])
    return np.sum(expanded, axis=1)

xs = np.linspace(1, 3, 100)

# Need to make this nicer, but for now this is fine
y1s = plot(xs[xs<=2].reshape(-1,1), lines[0][::-1])
y2s = plot(xs[xs>2].reshape(-1,1), lines[1][::-1])
y = np.concatenate([y1s, y2s])

plt.scatter(xs, y)
plt.scatter(raw_x, raw_y, c='tab:red')
plt.show()

In [ ]:
raw_x, raw_y = np.array([1,2,3]),  np.array([1,5,4])
n = len(raw_x)

A = np.zeros((4*(n-1), 4*(n-1)))
b = np.concatenate([raw_y[:1], np.repeat(raw_y[1:-1], 2), raw_y[-1:], np.zeros(2*(n-1))])

print(A)
print()
print(b)

In [ ]:
x, y = np.array([1,2,3]),  np.array([1,5,4])

n = len(x)

# The function constraints (each knot must be on the function)
xi = 0
for i in range(n-1):
    row = i*2
    col = i*4

    A[row][col:col+4] = [x[xi]**3, x[xi]**2, x[xi], 1]
    xi += 1
    A[row+1][col:col+4] = [x[xi]**3, x[xi]**2, x[xi], 1]

# Now the derivative constraints (1st + 2nd derivative at knots must be equal)
col = 0
xi = 0
for i in range(n-2):
    col = i*4
    row += 2
    xi += 1

    # 1st derivative
    A[row][col:col+3] = [3*(x[xi]**2), 2*x[xi], 1]
    A[row][col+4:col+7] = [-(3*(x[xi]**2)), -(2*x[xi]), -1]

    # 1st derivative
    A[row+1][col:col+2] = [6*(x[xi]), 2]
    A[row+1][col+4:col+6] = [-(6*(x[xi])), -2]

# Endpoint contraints. 2nd derivatives must be zero.
A[-2][:2] = [6*x[0], 2]
A[-1][-4:-2] = [6*x[-1], 2]

In [ ]:
x, y = np.array([1,2,3,4]),  np.array([1,5,4,5])

n = len(x)

b = np.array([1, 5, 5, 4, 4, 5, 0, 0, 0, 0, 0, 0], dtype=np.float64)
A = np.array([[1,1,1,1,0,0,0,0,0,0,0,0],
              [8,4,2,1,0,0,0,0,0,0,0,0], 
              [0,0,0,0,8,4,2,1,0,0,0,0],
              [0,0,0,0,27,9,3,1,0,0,0,0],
              [0,0,0,0,0,0,0,0,27,9,3,1],
              [0,0,0,0,0,0,0,0,64,16,4,1], # functions to here
              [12,4,1,0,-12,-4,-1,0,0,0,0,0],
              [12,2,0,0,-12,-2,0,0,0,0,0,0],
              [0,0,0,0,27,6,1,0,-27,-6,-1,0],
              [0,0,0,0,18,2,0,0,-18,-2,0,0], # derivatives to here
              [6,2,0,0,0,0,0,0,0,0,0,0],
              [0,0,0,0,0,0,0,0,24,2,0,0], # endpoints to here
              ], dtype=np.float64)

# The function constraints (each knot must be on the function)
xi = 0
for i in range(n-1):
    row = i*2
    col = i*4

    A[row][col:col+4] = [x[xi]**3, x[xi]**2, x[xi], 1]
    xi += 1
    A[row+1][col:col+4] = [x[xi]**3, x[xi]**2, x[xi], 1]

# Now the derivative constraints (1st + 2nd derivative at knots must be equal)
col = 0
xi = 0
for i in range(n-2):
    col = i*4
    row += 2
    xi += 1

    # 1st derivative
    A[row][col:col+3] = [3*(x[xi]**2), 2*x[xi], 1]
    A[row][col+4:col+7] = [-(3*(x[xi]**2)), -(2*x[xi]), -1]

    # 1st derivative
    A[row+1][col:col+2] = [6*(x[xi]), 2]
    A[row+1][col+4:col+6] = [-(6*(x[xi])), -2]

# Endpoint contraints. 2nd derivatives must be zero.
A[-2][:2] = [6*x[0], 2]
A[-1][-4:-2] = [6*x[-1], 2]

# Solving the system of equations
ws = np.dot(np.linalg.inv(A), b)
ws

In [ ]:
def predict(values, coeffs):
    if len(values) == 0:
        return np.empty(0)
    values = values.reshape(-1, 1)
    expanded = np.hstack([coeffs[i] * (values ** i) for i in range(0, len(coeffs))])
    return np.sum(expanded, axis=1)

x, y = np.array([1,2,3,4]),  np.array([1,5,4,5])
xs = np.linspace(x.min(), x.max(), 100)

ys = np.zeros(len(xs))
ws_pos = 0
knot = 1
insert_pos = 0

for i in range(n):
    # point before start endpoint
    if knot == 1:
        ys_to_add = predict(xs[xs<=knot], ws[ws_pos:ws_pos+4][::-1])
    # all other knots
    else:
        ys_to_add = predict(xs[np.logical_and(xs>knot-1, xs<=knot)], ws[ws_pos:ws_pos+4][::-1])
        ws_pos+=4

    ys[insert_pos:insert_pos + len(ys_to_add)] = ys_to_add
    insert_pos += len(ys_to_add)
    knot+=1

# point after endpoint endpoint
knot-=1
ws_pos-=4
ys_to_add = predict(xs[xs>knot], ws[ws_pos:ws_pos+4][::-1])
ys[insert_pos:insert_pos + len(ys_to_add)] = ys_to_add

plt.scatter(xs, ys)
plt.show()


In [ ]:
class cubic_spline():
    # --- Try and break this with tests -- 
    # actually fine as we take the avg closest values
    # check knots > 2

    def __init__(self, knots):
        self.knots = knots
        self.knot_xvals = None
        self.knot_yvals = None
        self.ws = None

    def fit(self, x, y, plot=False):
        if len(x) != len(y):
            raise ValueError('x and y must be of the same length')
        if len(x) <= self.knots:
            raise ValueError('len(x) must be greater than number of knots')

        # Sort values based on xs for knot calculation
        y = y[x.argsort()]
        x = np.sort(x)

        self.knot_xvals = np.linspace(x.min(), x.max(), num=self.knots+1)
        self.knot_yvals = self.find_knots(self.knot_xvals, x, y)
        
        self.ws = self.calc_piecewise_weights(self.knot_xvals, self.knot_yvals)

        if plot:
            self.plot_model(x, y)


    def calc_piecewise_weights(self, x, y):
        A = np.zeros((4*(self.knots), 4*(self.knots)))
        b = np.concatenate([y[:1], np.repeat(y[1:-1], 2), y[-1:], np.zeros(2*(self.knots))])

        # The function constraints (each knot must be on the function)
        xi = 0
        for i in range(self.knots):
            row = i*2
            col = i*4

            A[row][col:col+4] = [x[xi]**3, x[xi]**2, x[xi], 1]
            xi += 1
            A[row+1][col:col+4] = [x[xi]**3, x[xi]**2, x[xi], 1]

        # Now the derivative constraints (1st + 2nd derivative at knots must be equal)
        col = 0
        xi = 0
        for i in range(self.knots-1):
            col = i*4
            row += 2
            xi += 1

            # 1st derivative
            A[row][col:col+3] = [3*(x[xi]**2), 2*x[xi], 1]
            A[row][col+4:col+7] = [-(3*(x[xi]**2)), -(2*x[xi]), -1]

            # 1st derivative
            A[row+1][col:col+2] = [6*(x[xi]), 2]
            A[row+1][col+4:col+6] = [-(6*(x[xi])), -2]

        # Endpoint contraints. 2nd derivatives must be zero.
        A[-2][:2] = [6*x[0], 2]
        A[-1][-4:-2] = [6*x[-1], 2]

        # Solving the system of equations
        ws = np.dot(np.linalg.inv(A), b)
        return ws

    def predict(self, xs):
        ys = np.zeros(len(xs), dtype=np.float64) # int results in rounding errors

        ws_pos = 0
        insert_pos = 0

        for i in range(self.knots+1):
            # point before start endpoint
            if i == 0:
                ys_to_add = self.polynomial(xs[xs<=self.knot_xvals[i]], self.ws[ws_pos:ws_pos+4][::-1])
                ys[:len(ys_to_add)] = ys_to_add
            # all other knots
            else:
                ys_to_add = self.polynomial(xs[np.logical_and(xs>self.knot_xvals[i-1], xs<=self.knot_xvals[i])], self.ws[ws_pos:ws_pos+4][::-1])
                ws_pos+=4

            ys[insert_pos:insert_pos + len(ys_to_add)] = ys_to_add
            insert_pos += len(ys_to_add)

        # point after endpoint endpoint
        ws_pos-=4
        ys_to_add = self.polynomial(xs[xs>self.knot_xvals[i]], self.ws[ws_pos:ws_pos+4][::-1])
        ys[insert_pos:insert_pos + len(ys_to_add)] = ys_to_add
        return ys

    def plot_model(self, x, y):
        '''
        Plots the models hypothetical predictions, MSE, and true data points.
        '''
        # Plot hypothetical model values
        modelx = np.linspace(self.knot_xvals[0], self.knot_xvals[-1], 100)
        modely = self.predict(modelx)
        plt.plot(modelx, modely)

        # Plot data values
        plt.scatter(x, y)
        
        MSE = np.mean((y - self.predict(x)) ** 2)
        plt.title("{}, Knots: {}, MSE: {:.8f}".format(type(self).__name__, self.knots, MSE))

        for i in range(len(self.knot_xvals)):
            plt.axvline(x = self.knot_xvals[i], alpha=0.2)
        plt.show()

    @staticmethod
    def find_knots(knot_vals, xs, ys):
        knot_ys = np.zeros(len(knot_vals))
        for i, knot in enumerate(knot_vals):
            index = np.searchsorted(xs, int(knot))
            if int(knot) == knot:
                knot_ys[i] = ys[index]
            else:
                knot_ys[i] = np.mean(ys[index:index+2])
        return knot_ys

    @staticmethod
    def polynomial(values, coeffs):
        if len(values) == 0:
            return np.empty(0)
        values = values.reshape(-1, 1)
        expanded = np.hstack([coeffs[i] * (values ** i) for i in range(0, len(coeffs))])
        return np.sum(expanded, axis=1)

In [ ]:
x,y = generate_random_walk(100)

model = cubic_spline(knots=2)
model.fit(x, y, plot=True)

In [ ]:
x, y = np.array([1,2,3,4]),  np.array([1,5,4,5])

model = cubic_spline(knots=3)
model.fit(x, y, plot=True)

In [ ]:
x, y = np.array([1,2,3]),  np.array([1,5,4])

model = cubic_spline(knots=2)
model.fit(x, y, plot=True)

In [ ]:
def polynomial(values, coeffs):
    if len(values) == 0:
        return np.empty(0)
    values = values.reshape(-1, 1)
    expanded = np.hstack([coeffs[i] * (values ** i) for i in range(0, len(coeffs))])
    return np.sum(expanded, axis=1)

x, y = np.array([1,2,3,4]), np.array([1,5,4,5])

MAKE SURE TO MAKE DTYPE - np.float64
xs, ys = np.array([1,2,3,4], dtype=np.float64), np.array([1,5,4,5], dtype=np.float64)
xs = np.linspace(x.min()-1, x.max()+1, 100)
ys = np.zeros(len(xs), dtype=np.float64)

ws_pos = 0
insert_pos = 0

for i in range(model.knots+1):
    # point before start endpoint
    if i == 0:
        ys_to_add = polynomial(xs[xs<=model.knot_xvals[i]], model.ws[ws_pos:ws_pos+4][::-1])
        ys[:len(ys_to_add)] = ys_to_add
    # all other knots
    else:
        ys_to_add = polynomial(xs[np.logical_and(xs>model.knot_xvals[i-1], xs<=model.knot_xvals[i])], model.ws[ws_pos:ws_pos+4][::-1])
        ws_pos+=4

    ys[insert_pos:insert_pos + len(ys_to_add)] = ys_to_add
    insert_pos += len(ys_to_add)
    # print(ys)

# point after endpoint endpoint
ws_pos-=4
ys_to_add = polynomial(xs[xs>model.knot_xvals[i]], model.ws[ws_pos:ws_pos+4][::-1])
ys[insert_pos:insert_pos + len(ys_to_add)] = ys_to_add

plt.scatter(xs, ys)
plt.scatter(x, y, c='tab:red')
plt.show()